In [1]:
import seaborn as sns
import pandas as pd
import numpy as np

merging and joining are basically redundant and concatenating and appending are basically redundant

will explore merge and concatenating

In [4]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

tips = sns.load_dataset('tips')
tips.head(3)

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3


In [5]:
tips_bill = tips.groupby(['sex', 'smoker'])[['total_bill', 'tip']].sum()
tips_tip = tips.groupby(['sex', 'smoker'])[['total_bill', 'tip']].sum()

In [8]:
del tips_bill['tip']
del tips_tip['total_bill']

In [9]:
tips_bill

total_bill
sex    smoker            
Male   Yes        1337.07
       No         1919.75
Female Yes         593.27
       No          977.68

In [10]:
tips_tip

tip
sex    smoker        
Male   Yes     183.07
       No      302.00
Female Yes      96.74
       No      149.77

In [11]:
# 通过索引合并
pd.merge(tips_bill, tips_tip, right_index=True, left_index=True)

total_bill     tip
sex    smoker                    
Male   Yes        1337.07  183.07
       No         1919.75  302.00
Female Yes         593.27   96.74
       No          977.68  149.77

In [17]:
# 我们可以重置索引，然后在列上合并 - 可能是最简单的方式
pd.merge(tips_bill.reset_index(), tips_tip.reset_index(), on=['sex', 'smoker'])

,sex,smoker,total_bill,tip
0,Male,Yes,1337.07,183.07
1,Male,No,1919.75,302.00
2,Female,Yes,593.27,96.74
3,Female,No,977.68,149.77


In [18]:
# 可以自动识别，在所有相同列上进行join，但是建议不这样做
pd.merge(tips_bill.reset_index(), tips_tip.reset_index())

,sex,smoker,total_bill,tip
0,Male,Yes,1337.07,183.07
1,Male,No,1919.75,302.00
2,Female,Yes,593.27,96.74
3,Female,No,977.68,149.77


In [19]:
# 可以在一边指定列，一遍使用索引
pd.merge(tips_bill.reset_index(), tips_tip, left_on=['sex', 'smoker'], right_index=True)

,sex,smoker,total_bill,tip
0,Male,Yes,1337.07,183.07
1,Male,No,1919.75,302.00
2,Female,Yes,593.27,96.74
3,Female,No,977.68,149.77


In [28]:
# 可以做非常有趣的组合
tips_bill_strange = tips_bill.reset_index(level=0)
tips_bill_strange

,sex,total_bill
smoker,,
Yes,Male,1337.07
No,Male,1919.75
Yes,Female,593.27
No,Female,977.68


In [30]:
pd.merge(tips_tip.reset_index(), tips_bill_strange, on=['sex', 'smoker'])

,sex,smoker,tip,total_bill
0,Male,Yes,183.07,1337.07
1,Male,No,302.00,1919.75
2,Female,Yes,96.74,593.27
3,Female,No,149.77,977.68


In [31]:
pd.merge(tips_bill.reset_index(), tips_tip.reset_index().head(2), how='left')

,sex,smoker,total_bill,tip
0,Male,Yes,1337.07,183.07
1,Male,No,1919.75,302.00
2,Female,Yes,593.27,NaN
3,Female,No,977.68,NaN


In [32]:
pd.merge(tips_bill.reset_index(), tips_tip.reset_index().head(2), how='inner')

,sex,smoker,total_bill,tip
0,Male,Yes,1337.07,183.07
1,Male,No,1919.75,302.00


In [33]:
pd.merge(tips_bill.reset_index(), tips_tip.reset_index().head(2), how='outer', indicator=True)

,sex,smoker,total_bill,tip,_merge
0,Male,Yes,1337.07,183.07,both
1,Male,No,1919.75,302.00,both
2,Female,Yes,593.27,NaN,left_only
3,Female,No,977.68,NaN,left_only


In [34]:
pd.merge(tips_bill, tips_bill, right_index=True, left_index=True, suffixes=('_left', '_right'))

total_bill_left  total_bill_right
sex    smoker                                   
Male   Yes             1337.07           1337.07
       No              1919.75           1919.75
Female Yes              593.27            593.27
       No               977.68            977.68

concatenate 可以接收任意数量的数据帧，但是有特别的要求。所有传入的数据帧需要有相同的索引。所以不再是使用列作为索引了

In [37]:
pd.concat([tips_bill, tips_bill, tips_tip], sort=False)  # 按照行合并

total_bill     tip
sex    smoker                    
Male   Yes        1337.07     NaN
       No         1919.75     NaN
Female Yes         593.27     NaN
       No          977.68     NaN
Male   Yes        1337.07     NaN
       No         1919.75     NaN
Female Yes         593.27     NaN
       No          977.68     NaN
Male   Yes            NaN  183.07
       No             NaN  302.00
Female Yes            NaN   96.74
       No             NaN  149.77

In [38]:
pd.concat([tips_bill, tips_tip], axis=1)  # 按照列合并

total_bill     tip
sex    smoker                    
Male   Yes        1337.07  183.07
       No         1919.75  302.00
Female Yes         593.27   96.74
       No          977.68  149.77

In [41]:
# 最后，会在数据帧上加上其来自的地方
pd.concat([tips_bill, tips_tip], sort=False, keys=['num0', 'num1'], axis=0)  # 按照行排序

total_bill     tip
     sex    smoker                    
num0 Male   Yes        1337.07     NaN
            No         1919.75     NaN
     Female Yes         593.27     NaN
            No          977.68     NaN
num1 Male   Yes            NaN  183.07
            No             NaN  302.00
     Female Yes            NaN   96.74
            No             NaN  149.77